In [ ]:
!pip install fer dlib mediapipe SpeechRecognition opencv-python moviepy numpy scikit-learn tensorflow


In [ ]:
import os
import cv2
import numpy as np
import mediapipe as mp
import speech_recognition as sr
from fer import FER
from moviepy.editor import VideoFileClip
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras import layers, models


In [ ]:
from google.colab import files
import zipfile

# Upload the archive.zip file
uploaded = files.upload()

# Unzip the dataset
with zipfile.ZipFile('archive.zip', 'r') as zip_ref:
    zip_ref.extractall('dataset')

# Verify extraction
print("Extracted files:", os.listdir('dataset'))


In [ ]:
# Define paths to the 'confident' and 'unconfident' directories
confident_dir = 'dataset/sorted_data/sorted_data/confident'  # Adjust the path as necessary
unconfident_dir = 'dataset/sorted_data/sorted_data/unconfident'  # Adjust the path as necessary

# Function to load images and labels from a directory
def load_images_from_directory(directory, label):
    images = []
    labels = []
    for file_name in os.listdir(directory):
        file_path = os.path.join(directory, file_name)
        if file_name.lower().endswith(('.jpg', '.png')):
            image = cv2.imread(file_path)
            if image is not None:
                image = cv2.resize(image, (224, 224))  # Resize to a standard size
                images.append(image)
                labels.append(label)
    return images, labels

# Load images from both directories
confident_images, confident_labels = load_images_from_directory(confident_dir, label=1)
unconfident_images, unconfident_labels = load_images_from_directory(unconfident_dir, label=0)

# Combine the images and labels
all_images = np.array(confident_images + unconfident_images)
all_labels = np.array(confident_labels + unconfident_labels)

# Split the data into training and testing sets (80:20 split)
X_train, X_test, y_train, y_test = train_test_split(all_images, all_labels, test_size=0.2, random_state=42)

print(f"Training data shape: {X_train.shape}")
print(f"Testing data shape: {X_test.shape}")


Training data shape: (2302, 224, 224, 3)
Testing data shape: (576, 224, 224, 3)


In [ ]:
# Normalize the data
X_train = X_train / 255.0
X_test = X_test / 255.0

# Build a simple CNN model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

# Evaluate the model on test data
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_acc}")


  super().__init__(activity_regularizer=activity_regularizer, **kwargs)



Epoch 1/10
72/72 ━━━━━━━━━━━━━━━━━━━━ 272s 4s/step - accuracy: 0.6704 - loss: 0.6757 - val_accuracy: 0.8160 - val_loss: 0.4264
Epoch 2/10
72/72 ━━━━━━━━━━━━━━━━━━━━ 379s 5s/step - accuracy: 0.8418 - loss: 0.3894 - val_accuracy: 0.8142 - val_loss: 0.4460
Epoch 3/10
72/72 ━━━━━━━━━━━━━━━━━━━━ 351s 4s/step - accuracy: 0.8325 - loss: 0.3960 - val_accuracy: 0.8021 - val_loss: 0.4721
Epoch 4/10
72/72 ━━━━━━━━━━━━━━━━━━━━ 306s 4s/step - accuracy: 0.8484 - loss: 0.3596 - val_accuracy: 0.7899 - val_loss: 0.4327
Epoch 5/10
72/72 ━━━━━━━━━━━━━━━━━━━━ 273s 4s/step - accuracy: 0.8479 - loss: 0.3433 - val_accuracy: 0.8125 - val_loss: 0.4538
Epoch 6/10
72/72 ━━━━━━━━━━━━━━━━━━━━ 320s 4s/step - accuracy: 0.8624 - loss: 0.3319 - val_accuracy: 0.8090 - val_loss: 0.4223
Epoch 7/10
72/72 ━━━━━━━━━━━━━━━━━━━━ 325s 4s/step - accuracy: 0.8540 - loss: 0.3344 - val_accuracy: 0.8056 - val_loss: 0.4269
Epoch 8/10
72/72 ━━━━━━━━━━━━━━━━━━━━ 326s 4s/step - accuracy: 0.8596 - loss: 0.3266 - val_accuracy: 0.8125 - v

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv3D, MaxPooling3D, Flatten, Dense

# Define the 3D CNN model
model = Sequential()

# 3D Convolutional layer to handle 114 frames of size 128x128
model.add(Conv3D(32, kernel_size=(3, 3, 3), activation='relu', input_shape=(114, 128, 128, 3)))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Conv3D(64, kernel_size=(3, 3, 3), activation='relu'))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Conv3D(128, kernel_size=(3, 3, 3), activation='relu'))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Flatten())

# Dense layers
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))  # Binary classification: Confident or Not Confident

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print model summary to verify architecture
model.summary()


  super().__init__(activity_regularizer=activity_regularizer, **kwargs)



Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv3d (Conv3D)                      │ (None, 112, 126, 126, 32)   │           2,624 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling3d (MaxPooling3D)         │ (None, 56, 63, 63, 32)      │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv3d_1 (Conv3D)                    │ (None, 54, 61, 61, 64)      │          55,360 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling3d_1 (MaxPooling3D)       │ (None, 27, 30, 30, 64)      │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv3d_2 (Conv3D)                    │ (None, 25, 28, 28, 128)     │         221,312 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling3d_2 (MaxPooling3D)       │ (None, 12, 14, 14, 128)     │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 301056)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 256)                 │      77,070,592 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │             257 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 77,350,145 (295.07 MB)

 Trainable params: 77,350,145 (295.07 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
def analyze_video_confidence(video_path, model):
    cap = cv2.VideoCapture(video_path)
    frames = []

    while cap.isOpened() and len(frames) < 114:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (128, 128))  # Resize to (128, 128)
        frame = frame / 255.0
        frames.append(frame)

    cap.release()

    while len(frames) < 114:
        frames.append(np.zeros((128, 128, 3)))

    frames = np.array(frames)
    frames = np.expand_dims(frames, axis=0)  # Shape: (1, 114, 128, 128, 3)

    # Predict using the trained model
    predictions = model.predict(frames)
    avg_prediction = np.mean(predictions)
    confidence_label = "Confident" if avg_prediction >= 0.5 else "Not Confident"

    return confidence_label

# Upload and analyze the video
uploaded_video = files.upload()
video_path = list(uploaded_video.keys())[0]
result = analyze_video_confidence(video_path, model)
if result == "Confident":
    print(f"The subject in the video is: {result} and honest.")
else:
    print(f"The subject in the video is: {result} and cheating.")


In [2]:
!git clone https://github.com/hrutujamirgal/interview_prep.git


Cloning into 'interview_prep'...
remote: Enumerating objects: 58, done.
remote: Counting objects: 100% (58/58), done.
remote: Compressing objects: 100% (52/52), done.
remote: Total 58 (delta 1), reused 54 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (58/58), 16.52 MiB | 33.56 MiB/s, done.
Resolving deltas: 100% (1/1), done.


In [9]:
# Move the current notebook to the models folder
!mv /content/traintestemotionconfidencemodel.ipynb /content/interview_prep/models/


mv: cannot stat '/content/traintestemotionconfidencemodel.ipynb': No such file or directory


In [10]:
%cd /content/interview_prep/

/content/interview_prep


In [11]:
!git add models/traintestemotionconfidencemodel.ipynb

fatal: pathspec 'models/traintestemotionconfidencemodel.ipynb' did not match any files
